In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
from bokeh.palettes import RdBu
from bokeh.models import LinearColorMapper, ColumnDataSource, ColorBar
from bokeh.models.ranges import FactorRange
from bokeh.plotting import figure, show, output_file, save
from bokeh.io import output_notebook
from bokeh.transform import transform


import numpy as np
import pandas as pd

In [2]:
data = pd.read_excel('data/mental_WHO.xlsx')

In [3]:
data.head()

,Country,Year,ICD rev and list,Cause details,Sex,Format,Sum of selected ages,< 1 year,1 year,2 years,...,55 - 59,60 - 64,65 - 69,70 - 74,75 - 79,80 - 84,85 - 89,90 - 95,95 +,Unknown
0,Denmark,2015,104,F010,F,0,1,0,0,0.0,...,0,1,0,0,0,0,0,0.0,0.0,0
1,Denmark,2015,104,F010,M,0,1,0,0,0.0,...,0,0,0,0,0,1,0,0.0,0.0,0
2,Denmark,2015,104,F011,M,0,15,0,0,0.0,...,0,0,2,1,3,2,3,3.0,1.0,0
3,Denmark,2015,104,F011,F,0,24,0,0,0.0,...,0,0,0,3,2,3,7,8.0,1.0,0
4,Denmark,2015,104,F012,F,0,4,0,0,0.0,...,0,0,0,0,0,2,1,1.0,0.0,0


In [4]:
datadr = data.groupby(['Country', 'Year', 'Sex', 'Sum of selected ages'], as_index=False).mean()

In [5]:
# datadr.fillna(0, inplace=True)

In [7]:
cols = ['Country', 'Year', 'Sex', 'Sum of selected ages']
mhd = datadr[cols]

In [11]:
males = mhd[mhd['Sex'] == 'M']
females = mhd[mhd['Sex'] == 'F']

In [12]:
males.shape

(902, 4)

In [13]:
males = males.groupby(['Country', 'Year', 'Sex'], as_index = False).mean()
females = females.groupby(['Country', 'Year', 'Sex'], as_index = False).mean()
females = females[(females['Year'] != 1994) & (females['Year'] != 1995) & (females['Year'] != 1996) & (females['Year'] != 2016)]
males = males[(males['Year'] != 1994) & (males['Year'] != 1995) & (males['Year'] != 1996) & (males['Year'] != 2016)]

In [14]:
males = males.groupby(['Country', 'Year', 'Sex'], as_index = False).mean()
females = females.groupby(['Country', 'Year', 'Sex'], as_index = False).mean()

In [15]:
females = females[(females['Year'] != 1994) & (females['Year'] != 1995) & (females['Year'] != 1996) & (females['Year'] != 2016)]
males = males[(males['Year'] != 1994) & (males['Year'] != 1995) & (males['Year'] != 1996) & (males['Year'] != 2016)]

In [16]:
males[['Year']] = males[['Year']].astype('object')

In [17]:
males.dtypes

Country                  object
Year                     object
Sex                      object
Sum of selected ages    float64
dtype: object

In [18]:
males['size'] = males['Sum of selected ages']

In [19]:
years = []
for i in range(len(males['Year'].tolist())):
    years.append(str(males['Year'].tolist()[i]))

In [20]:
males['Year_'] = years

In [21]:
pop = pd.read_excel('data/population.xlsx')

In [22]:
pop_v = pop.groupby(['Country', 'Year'], as_index=False).mean()
pop_to_unite = pop_v[['Country', 'Year', 'Sum of selected ages']]

In [23]:
pop_to_unite.fillna(pop_to_unite['Sum of selected ages'].mean(), inplace=True)

/home/tory/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:4153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [24]:
pop_to_unite.isna().sum()

Country                 0
Year                    0
Sum of selected ages    0
dtype: int64

In [25]:
males = mhd[mhd['Sex'] == 'M']
females = mhd[mhd['Sex'] == 'F']

In [26]:
males = males.groupby(['Country', 'Year', 'Sex'], as_index = False).mean()
females = females.groupby(['Country', 'Year', 'Sex'], as_index = False).mean()
females = females[(females['Year'] != 1994) & (females['Year'] != 1995) & (females['Year'] != 1996) & (females['Year'] != 2016)]
males = males[(males['Year'] != 1994) & (males['Year'] != 1995) & (males['Year'] != 1996) & (males['Year'] != 2016)]

In [27]:
years = []
for i in range(len(males['Year'].tolist())):
    years.append(str(males['Year'].tolist()[i]))
males['Year'] = years

In [28]:
years = []
for i in range(len(females['Year'].tolist())):
    years.append(str(females['Year'].tolist()[i]))
females['Year'] = years

In [29]:
years = []
for i in range(len(pop_to_unite['Year'].tolist())):
    years.append(str(pop_to_unite['Year'].tolist()[i]))
pop_to_unite['Year'] = years

/home/tory/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [30]:
extended_data_f = pd.merge(females, pop_to_unite, left_on = ['Country', 'Year'], right_on = ['Country', 'Year'])
extended_data_f.head()

,Country,Year,Sex,Sum of selected ages_x,Sum of selected ages_y
0,Denmark,1997,F,61.909091,4.799444e+06
1,Denmark,1998,F,64.750000,4.799444e+06
2,Denmark,1999,F,72.000000,4.799444e+06
3,Denmark,2000,F,79.000000,5.337344e+06
4,Denmark,2001,F,70.785714,5.355082e+06


In [31]:
extended_data_m = pd.merge(males, pop_to_unite, left_on = ['Country', 'Year'], right_on = ['Country', 'Year'])
extended_data_m.head()

,Country,Year,Sex,Sum of selected ages_x,Sum of selected ages_y
0,Denmark,1997,M,46.363636,4.799444e+06
1,Denmark,1998,M,50.090909,4.799444e+06
2,Denmark,1999,M,64.363636,4.799444e+06
3,Denmark,2000,M,54.692308,5.337344e+06
4,Denmark,2001,M,63.750000,5.355082e+06


In [32]:
extended_data_f['Val_by_pop'] = extended_data_f['Sum of selected ages_x'] / extended_data_f['Sum of selected ages_y']
extended_data_m['Val_by_pop'] = extended_data_m['Sum of selected ages_x'] / extended_data_m['Sum of selected ages_y']

In [33]:
extended_data_f['size_'] = extended_data_f['Val_by_pop']*2000**2
extended_data_m['size_'] = extended_data_m['Val_by_pop']*2000**2

In [36]:
! mkdir html_output

In [37]:
# output_notebook()


colors = ["#fffcad", "#fffa85", "#fff400", "#f9e200", "#f0da00", "#f0ce00", "#f0c200"]
mapper = LinearColorMapper(palette=colors,
                           low=extended_data_f['Val_by_pop'].min(), high=extended_data_f['Val_by_pop'].max())


colors = list(reversed(RdBu[9]))
exp_cmap = LinearColorMapper(palette=colors, 
                             low = -1, 
                             high = 1)



p2 = figure(x_range = FactorRange(), y_range = FactorRange(), plot_width=1200, 
           plot_height=900, title="Female mortality from mental disorders (Northern countries)",
           toolbar_location=None, tools="hover",
          background_fill_color = "#000000")

p2.title.text_font_size = '25pt' #heading

#axis labels sizes
p2.xaxis.axis_label_text_font_size = "20pt"
p2.yaxis.axis_label_text_font_size = "20pt"

#the categories size
p2.xaxis.major_label_text_font_size = "13pt" 
p2.yaxis.major_label_text_font_size = "13pt"


p2.scatter("Country", "Year", source=extended_data_f, fill_alpha=1,  line_width=0, size="size_", 
          fill_color=transform('Val_by_pop', mapper), line_color="#FFFFFF", alpha = 0.9)

p2.x_range.factors = sorted(extended_data_f['Country'].unique().tolist())
p2.y_range.factors = sorted(extended_data_f['Year'].unique().tolist(), reverse = True)

p2.xaxis.axis_label = 'Country'
p2.yaxis.axis_label = 'Year'

# show(p2)


In [38]:
output_file("html_output/Death_from_mental_disoders_WHO_females.html")
save(p2)

'/home/tory/Desktop/WHO/code_and_data/html_output/Death_from_mental_disoders_WHO_females.html'